In [2]:
import numpy as np
import pandas as pd

train_file = pd.read_csv('train.csv')
train_file.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# colunas utilizáveis
y = train_file[['Survived']]
x = train_file[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]

print(x.head())

   Pclass     Sex   Age  SibSp  Parch Embarked
0       3    male  22.0      1      0        S
1       1  female  38.0      1      0        C
2       3  female  26.0      0      0        S
3       1  female  35.0      1      0        S
4       3    male  35.0      0      0        S


In [5]:
#colunas faltando dado
cols_with_missing = x.columns[x.isnull().any()]

print(cols_with_missing)

Index(['Age', 'Embarked'], dtype='object')


In [9]:
# tratamento de NaN
x_nan = x.fillna(0)

cols_with_missing2 = x_nan.columns[x_nan.isnull().any()]
print(cols_with_missing2)

print(x_nan.head())

Index([], dtype='object')
   Pclass     Sex   Age  SibSp  Parch Embarked
0       3    male  22.0      1      0        S
1       1  female  38.0      1      0        C
2       3  female  26.0      0      0        S
3       1  female  35.0      1      0        S
4       3    male  35.0      0      0        S


In [10]:
x2 = x_nan.copy()

#add number of memeber families including the one 
x2['family_number'] = x_nan.SibSp + x_nan.Parch + 1

print(x2.head())

   Pclass     Sex   Age  SibSp  Parch Embarked  family_number
0       3    male  22.0      1      0        S              2
1       1  female  38.0      1      0        C              2
2       3  female  26.0      0      0        S              1
3       1  female  35.0      1      0        S              2
4       3    male  35.0      0      0        S              1


In [11]:
# transformar em variaveis de categoria
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_x = x2.copy()
sex_label_encoder = LabelEncoder()
embarked_label_encoder = LabelEncoder()
label_x['Sex'] = sex_label_encoder.fit_transform(x2['Sex'].astype(str))
label_x['Embarked'] = embarked_label_encoder.fit_transform(x2['Embarked'].astype(str))

print(label_x.head())

   Pclass  Sex   Age  SibSp  Parch  Embarked  family_number
0       3    1  22.0      1      0         3              2
1       1    0  38.0      1      0         1              2
2       3    0  26.0      0      0         3              1
3       1    0  35.0      1      0         3              2
4       3    1  35.0      0      0         3              1


In [14]:
hot_enconder = OneHotEncoder(handle_unknown='ignore', sparse=False)

cols_ohe = pd.DataFrame( hot_enconder.fit_transform(label_x[['Sex', 'Embarked']] ))
cols_ohe.index = label_x.index

print(cols_ohe.head())


     0    1    2    3    4    5
0  0.0  1.0  0.0  0.0  0.0  1.0
1  1.0  0.0  0.0  1.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  1.0
3  1.0  0.0  0.0  0.0  0.0  1.0
4  0.0  1.0  0.0  0.0  0.0  1.0


In [15]:
x_OH = label_x.drop(['Sex', 'Embarked'], axis=1) 
print(x_OH.head())

x_OH = pd.concat([x_OH, cols_ohe], axis=1)
print(x_OH.head())


   Pclass   Age  SibSp  Parch  family_number
0       3  22.0      1      0              2
1       1  38.0      1      0              2
2       3  26.0      0      0              1
3       1  35.0      1      0              2
4       3  35.0      0      0              1
   Pclass   Age  SibSp  Parch  family_number    0    1    2    3    4    5
0       3  22.0      1      0              2  0.0  1.0  0.0  0.0  0.0  1.0
1       1  38.0      1      0              2  1.0  0.0  0.0  1.0  0.0  0.0
2       3  26.0      0      0              1  1.0  0.0  0.0  0.0  0.0  1.0
3       1  35.0      1      0              2  1.0  0.0  0.0  0.0  0.0  1.0
4       3  35.0      0      0              1  0.0  1.0  0.0  0.0  0.0  1.0


In [38]:

from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_OH)

In [39]:
#SVM
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

steps = [ ('SVC', SVC())]
param_grid = {'SVC__C': [1, 10, 100, 200], 
             'SVC__kernel' : ['rbf', 'linear']}

estimator = Pipeline(steps=steps)
cv = GridSearchCV(estimator, param_grid, verbose=5, n_jobs=-1, cv=5)
cv.fit(x_train, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.1min finished
C:\Users\igorl\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('SVC',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'SVC__C': [1, 10, 100, 200],
                         'SVC__kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 

In [40]:

print(cv.best_params_, cv.best_score_)

{'SVC__C': 1, 'SVC__kernel': 'rbf'} 0.8170594837261503


In [20]:
#TESTE
teste_file = pd.read_csv('test.csv')
print(teste_file.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


In [41]:
passengerID = pd.DataFrame(teste_file.PassengerId)
x_test = teste_file[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]

In [42]:
x_test.fillna(0, inplace=True)
x_test['family_number'] = x_test.SibSp + x_test.Parch + 1

print(x_test.head())
x_test['Sex'] = sex_label_encoder.transform(x_test['Sex'].astype(str))
x_test['Embarked'] = embarked_label_encoder.transform(x_test['Embarked'].astype(str))

test_cols_ohe = pd.DataFrame( hot_enconder.transform(x_test[['Sex', 'Embarked']] ))
test_cols_ohe.index = x_test.index
x_test = x_test.drop(['Sex', 'Embarked'], axis=1) 
x_test = pd.concat([x_test, test_cols_ohe], axis=1)

print(x_test.head())
x_test =  sc_X.transform(x_test)

   Pclass     Sex   Age  SibSp  Parch Embarked  family_number
0       3    male  34.5      0      0        Q              1
1       3  female  47.0      1      0        S              2
2       2    male  62.0      0      0        Q              1
3       3    male  27.0      0      0        S              1
4       3  female  22.0      1      1        S              3
   Pclass   Age  SibSp  Parch  family_number    0    1    2    3    4    5
0       3  34.5      0      0              1  0.0  1.0  0.0  0.0  1.0  0.0
1       3  47.0      1      0              2  1.0  0.0  0.0  0.0  0.0  1.0
2       2  62.0      0      0              1  0.0  1.0  0.0  0.0  1.0  0.0
3       3  27.0      0      0              1  0.0  1.0  0.0  0.0  0.0  1.0
4       3  22.0      1      1              3  1.0  0.0  0.0  0.0  0.0  1.0


C:\Users\igorl\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\igorl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\igorl\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ""

In [45]:

pred = pd.DataFrame( cv.predict(sc_X.transform(x_test)))

print(pred.head())

   0
0  0
1  1
2  0
3  0
4  1


In [63]:
print(passengerID.shape)
print( pred.shape)

submission = pd.concat([passengerID, pred], axis=1)
submission.columns = ['PassengerId', 'Survived']

print(submission.head(), submission.shape)

submission.to_csv('submission.csv', index=False)

(418, 1)
(418, 1)
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1 (418, 2)
